<a href="https://colab.research.google.com/github/FernandaSzAdr/Redes-Neurais--UFRPE-2019.1/blob/master/MLP%20-%20Multilayer%20Perceptron/codigos/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/FernandaSzAdr/Redes-Neurais--UFRPE-2019.1

Cloning into 'Redes-Neurais--UFRPE-2019.1'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 162 (delta 48), reused 162 (delta 48), pack-reused 0
Receiving objects: 100% (162/162), 21.70 MiB | 43.39 MiB/s, done.
Resolving deltas: 100% (48/48), done.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [2]:
!pip install ipdb
import random
import numpy as np

  Stored in directory: /root/.cache/pip/wheels/59/24/91/695211bd228d40fb22dff0ce3f05ba41ab724ab771736233f3
Successfully built ipdb


A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [0]:
def funcao_ativacao(tipo, is_prime, z):
  # Função de Ativação Sigmóide
  def sigmoid(z):
      return 1.0/(1.0+np.exp(-z))

  def relu(z):
      return np.array([[max(0, i[0])] for i in z])

  # Função para retornar as derivadas da função Sigmóide
  def sigmoid_prime(z):
      return sigmoid(z)*(1-sigmoid(z))

  def relu_prime(z):
      return relu(z)*(1-relu(z))
  
  switcher = {
      'sigmoid': sigmoid,
      'relu': relu
  }
  
  switcher_prime = {
      'sigmoid': sigmoid_prime,
      'relu': relu_prime,
  }
  
  if is_prime:
    fun = switcher_prime.get(tipo)
  else:
    fun = switcher.get(tipo)
  return fun(z)


def atualiza_taxa_aprend(taxa_inicial, epoch, n_epochs):
    return taxa_inicial * np.exp(-epoch / n_epochs)

In [0]:
# Classe Network
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a, tipo_fun):
        """Retorna a saída da rede se `a` for entrada."""
        for b, w in zip(self.biases, self.weights):
            a = funcao_ativacao(tipo_fun, False, np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None, tipo_fun='sigmoid'):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
        
        eta_inicial = eta
        training_data = list(training_data)
        n = len(training_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, tipo_fun)
                        
            eta = atualiza_taxa_aprend(eta_inicial, j, epochs)
        
          
        if test_data:
            predic = self.evaluate(test_data, tipo_fun)
            print("Total Input {} / Total predict {}".format(len(test_data), predic))
            return predic
        else:
            print("Epoch {} finalizada".format(j))
               
    def update_mini_batch(self, mini_batch, eta, tipo_fun):
        """Atualiza os pesos e bias da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `eta` é a taxa de aprendizado."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y, tipo_fun)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y, tipo_fun):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo C_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores z, camada por camada
        zs = [] 

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = funcao_ativacao(tipo_fun, False, z)
            activations.append(activation)
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * funcao_ativacao(tipo_fun, True, zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante. 
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = funcao_ativacao(tipo_fun, True, z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data, tipo_fun):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""

        test_results = [(np.argmax(self.feedforward(x, tipo_fun)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [0]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('Redes-Neurais--UFRPE-2019.1/MLP - Multilayer Perceptron/codigos/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def all_data():
  training_data, validation_data, test_data = load_data_wrapper()

  training_data = list(training_data)
  validation_data = list(validation_data)
  test_data = list(test_data)

  return training_data + validation_data + test_data

# Mini-Projeto part. 1

1) Realizar avaliaçao usando validação cruzada

2) Modificar a taxa de aprendizagem pra diminuir com o tempo

3) Realizar avaliações com a validação cruzada modificando os seguintes parâmetros:
     
     a) Taxa de aprendizagem: 0.5, 0.1 e 0.001
     b) Função de ativação RELU
     c) 3 diferentes configurações de neurônios na camada  intermediária


Data de Entrega: 23/04/2019

In [0]:
import pandas as pd
from sklearn.model_selection import KFold

In [0]:
data = all_data()

In [0]:
media_modelos = {}
funcao_ativacao_list = []
rede = []
taxa_aprendizagem = []
medias = []

epoch = 10
n_k = 4

## Rede 1 -> Configuração 784-30-10

In [0]:
tamanho = [784, 30, 10]

### Função ativação - Relu

#### Taxa de aprendizagem - 0.5

In [38]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in add


Total Input 17500 / Total predict 1852
K 1
Total Input 17500 / Total predict 1780
K 2
Total Input 17500 / Total predict 1543
K 3
Total Input 17500 / Total predict 0


In [39]:
teste1 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste1

,K,Acertos
0,0,1852
1,1,1780
2,2,1543
3,3,0


In [40]:
media = teste1['Acertos'].mean()

media

1293.75

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [42]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [43]:
teste2 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste2

,K,Acertos
0,0,1741
1,1,1693
2,2,3998
3,3,17500


In [44]:
media = teste2['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [46]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [47]:
teste3 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste3

,K,Acertos
0,0,1741
1,1,1693
2,2,3998
3,3,17500


In [48]:
media = teste3['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

### Função ativação - Sigmoid

#### Taxa de aprendizagem - 0.5

In [50]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

K 0
Total Input 17500 / Total predict 4606
K 1
Total Input 17500 / Total predict 3471
K 2
Total Input 17500 / Total predict 3424
K 3
Total Input 17500 / Total predict 1837


In [51]:
teste4 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste4

,K,Acertos
0,0,4606
1,1,3471
2,2,3424
3,3,1837


In [52]:
media = teste4['Acertos'].mean()

media

3334.5

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [54]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

K 0
Total Input 17500 / Total predict 902
K 1
Total Input 17500 / Total predict 4794
K 2
Total Input 17500 / Total predict 5085
K 3
Total Input 17500 / Total predict 1898


In [55]:
teste5 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste5

,K,Acertos
0,0,902
1,1,4794
2,2,5085
3,3,1898


In [56]:
media = teste5['Acertos'].mean()

media

3169.75

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [58]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

K 0
Total Input 17500 / Total predict 1648
K 1
Total Input 17500 / Total predict 1727
K 2
Total Input 17500 / Total predict 1562
K 3
Total Input 17500 / Total predict 0


In [59]:
teste6 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste6

,K,Acertos
0,0,1648
1,1,1727
2,2,1562
3,3,0


In [60]:
media = teste6['Acertos'].mean()

media

1234.25

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 1')
taxa_aprendizagem.append(at)
medias.append(media)

## Rede 2 -> Configuração 784-50-10

In [0]:
tamanho = [784, 50, 10]

### Função ativação - Relu

#### Taxa de aprendizagem - 0.5

In [63]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


Total Input 17500 / Total predict 1829
K 1
Total Input 17500 / Total predict 1747
K 2
Total Input 17500 / Total predict 1525
K 3
Total Input 17500 / Total predict 0


In [64]:
teste7 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste7

,K,Acertos
0,0,1829
1,1,1747
2,2,1525
3,3,0


In [65]:
media = teste7['Acertos'].mean()

media

1275.25

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [67]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [68]:
teste8 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste8

,K,Acertos
0,0,1741
1,1,1693
2,2,3998
3,3,17500


In [69]:
media = teste8['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [71]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [72]:
teste9 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste9

,K,Acertos
0,0,1741
1,1,1693
2,2,3998
3,3,17500


In [73]:
media = teste9['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

### Função ativação - Sigmoid

#### Taxa de aprendizagem - 0.5

In [75]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

K 0
Total Input 17500 / Total predict 2428
K 1
Total Input 17500 / Total predict 1636
K 2
Total Input 17500 / Total predict 1561
K 3
Total Input 17500 / Total predict 2


In [76]:
teste10 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste10

,K,Acertos
0,0,2428
1,1,1636
2,2,1561
3,3,2


In [77]:
media = teste10['Acertos'].mean()

media

1406.75

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [79]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

K 0
Total Input 17500 / Total predict 2225
K 1
Total Input 17500 / Total predict 1998
K 2
Total Input 17500 / Total predict 1384
K 3
Total Input 17500 / Total predict 0


In [80]:
teste11 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste11

,K,Acertos
0,0,2225
1,1,1998
2,2,1384
3,3,0


In [81]:
media = teste11['Acertos'].mean()

media

1401.75

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [12]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

K 0
Total Input 17500 / Total predict 1373
K 1
Total Input 17500 / Total predict 1473
K 2
Total Input 17500 / Total predict 1858
K 3
Total Input 17500 / Total predict 4013


In [13]:
teste12 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste12

,K,Acertos
0,0,1373
1,1,1473
2,2,1858
3,3,4013


In [14]:
media = teste12['Acertos'].mean()

media

2179.25

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 2')
taxa_aprendizagem.append(at)
medias.append(media)

## Rede 3 -> Configuração 784-15-10

In [0]:
tamanho = [784, 15, 10]

### Função ativação - Relu

#### Taxa de aprendizagem - 0.5

In [17]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply


Total Input 17500 / Total predict 1741
K 1
Total Input 17500 / Total predict 1693
K 2
Total Input 17500 / Total predict 3998
K 3
Total Input 17500 / Total predict 17500


In [18]:
teste7 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste7

,K,Acertos
0,0,1741
1,1,1693
2,2,3998
3,3,17500


In [19]:
media = teste7['Acertos'].mean()

media

6233.0

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [0]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

K 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in multiply
  


In [0]:
teste8 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste8

In [0]:
media = teste8['Acertos'].mean()

media

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [0]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='relu')
  acertos_dict[k] = result

In [0]:
teste9 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste9

In [0]:
media = teste9['Acertos'].mean()

media

In [0]:
funcao_ativacao_list.append('relu')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

### Função ativação - Sigmoid

#### Taxa de aprendizagem - 0.5

In [0]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.5

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

In [0]:
teste10 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste10

In [0]:
media = teste10['Acertos'].mean()

media

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.1

In [0]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.1

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

In [0]:
teste11 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste11

In [0]:
media = teste11['Acertos'].mean()

media

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

#### Taxa de aprendizagem - 0.001

In [0]:
net = Network(tamanho)
acertos_dict = {}
kf = KFold(n_splits=n_k)
at = 0.001

for i, index in enumerate(kf.split(data)):
  train_index = index[0]
  test_index = index[1]
  
  k = "K {}".format(i)
  
  print(k)
  training_data = data[train_index[0]:train_index[-1]+1]
  testing_data = data[test_index[0]:test_index[-1]+1]
  
  result = net.SGD(training_data, epoch, 10, at,test_data=testing_data, tipo_fun='sigmoid')
  acertos_dict[k] = result

In [0]:
teste12 = pd.DataFrame({
    'K': np.arange(kf.get_n_splits(data)),
    'Acertos': list(acertos_dict.values())
})

teste12

In [0]:
media = teste12['Acertos'].mean()

media

In [0]:
funcao_ativacao_list.append('sigmoid')
rede.append('rede 3')
taxa_aprendizagem.append(at)
medias.append(media)

## Resultados 1 Entrega

In [0]:
resultado_total = pd.DataFrame({
    'rede': rede,
    'funcao de ativacao': funcao_ativacao_list,
    'taxa de aprendizagem': taxa_aprendizagem,
    'media k-fold': medias
})

resultado_total

In [0]:
resultado_total['media k-fold'].max()

# Mini-Projeto part. 2

A partir dos melhores resultados obtidos na etapa anterior, resolver as seguintes questões:

4) Implementar as regularizações L1 e L2

5) Implementar o Momento

6) Comparar os experimentos e explicar o que achou dos resultados.

Data de Entrega: 30/04/2019